In [21]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
%autoreload 2

In [23]:
from PIL import Image
import numpy as np
import torch
from torchvision import transforms

In [24]:
from importlib import reload
import pytorch_pretrained_vit
reload(pytorch_pretrained_vit)

<module 'pytorch_pretrained_vit' from '/home/luke/projects/experiments/ViT-PyTorch/pytorch_pretrained_vit/__init__.py'>

In [25]:
name = 'L_32'
filename = 'jax_weights/ViT-L_32.npz'
# npz_files = {
#     'B_16': 'jax_weights/ViT-B_16.npz',
#     'B_32': 'jax_weights/ViT-B_32.npz',
#     'L_16': 'jax_weights/ViT-L_16.npz',
#     'L_32': 'jax_weights/ViT-L_32.npz',
#     'B_16_imagenet1k': 'jax_weights/ViT-B_16_imagenet1k.npz',
#     'B_32_imagenet1k': 'jax_weights/ViT-B_32_imagenet1k.npz',
#     'L_16_imagenet1k': 'jax_weights/ViT-L_16_imagenet1k.npz',
#     'L_32_imagenet1k': 'jax_weights/ViT-L_32_imagenet1k.npz',
# }
num_classes = 21843
# num_classes = 1000

In [26]:
def jax_to_pytorch(k):
    k = k.replace('Transformer/encoder_norm', 'norm')
    k = k.replace('LayerNorm_0', 'norm1')
    k = k.replace('LayerNorm_2', 'norm2')
    k = k.replace('MlpBlock_3/Dense_0', 'pwff.fc1')
    k = k.replace('MlpBlock_3/Dense_1', 'pwff.fc2')
    k = k.replace('MultiHeadDotProductAttention_1/out', 'proj')
    k = k.replace('MultiHeadDotProductAttention_1/query', 'attn.proj_q')
    k = k.replace('MultiHeadDotProductAttention_1/key', 'attn.proj_k')
    k = k.replace('MultiHeadDotProductAttention_1/value', 'attn.proj_v')
    k = k.replace('Transformer/posembed_input', 'positional_embedding')
    k = k.replace('encoderblock_', 'blocks.')
    k = 'patch_embedding.bias' if k == 'embedding/bias' else k
    k = 'patch_embedding.weight' if k == 'embedding/kernel' else k
    k = 'class_token' if k == 'cls' else k
    k = k.replace('head', 'fc')
    k = k.replace('kernel', 'weight')
    k = k.replace('scale', 'weight')
    k = k.replace('/', '.')
    k = k.lower()
    return k


def convert(npz, state_dict):
    new_state_dict = {}
    pytorch_k2v = {jax_to_pytorch(k): v for k, v in npz.items()}
    for pytorch_k, pytorch_v in state_dict.items():
        
        # Naming
        if 'self_attn.out_proj.weight' in pytorch_k:
            v = pytorch_k2v[pytorch_k]
            v = v.reshape(v.shape[0] * v.shape[1], v.shape[2])
        elif 'self_attn.in_proj_' in pytorch_k:
            v = np.stack((pytorch_k2v[pytorch_k + '*q'], 
                          pytorch_k2v[pytorch_k + '*k'], 
                          pytorch_k2v[pytorch_k + '*v']), axis=0)
        else:
            if pytorch_k not in pytorch_k2v:
                print(pytorch_k, list(pytorch_k2v.keys()))
                assert False
            v = pytorch_k2v[pytorch_k]
        v = torch.from_numpy(v)
        
        # Sizing
        if '.weight' in pytorch_k:
            if len(pytorch_v.shape) == 2:
                v = v.transpose(0, 1)
            if len(pytorch_v.shape) == 4:
                v = v.permute(3, 2, 0, 1)
        if ('proj.weight' in pytorch_k):
            v = v.transpose(0, 1)
            v = v.reshape(-1, v.shape[-1]).T
        if ('attn.proj_' in pytorch_k and 'weight' in pytorch_k):
            v = v.permute(0, 2, 1)
            v = v.reshape(-1, v.shape[-1])
        if 'attn.proj_' in pytorch_k and 'bias' in pytorch_k:
            v = v.reshape(-1)
        new_state_dict[pytorch_k] = v
    return new_state_dict


def check_model(model, name):
    model.eval()
    img = Image.open('../examples/simple/img.jpg')
    img = transforms.Compose([transforms.Resize(model.image_size), transforms.ToTensor(), transforms.Normalize(0.5, 0.5)])(img).unsqueeze(0)
    if 'imagenet1k' in name:
        labels_file = '../examples/simple/labels_map.txt' 
        labels_map = json.load(open(labels_file))
        labels_map = [labels_map[str(i)] for i in range(1000)]
        print('-----\nShould be index 388 (panda) w/ high probability:')
    else:
        labels_map = open('../examples/simple/labels_map_21k.txt').read().splitlines()
    with torch.no_grad():
        outputs = model(img).squeeze(0)
    for idx in torch.topk(outputs, k=3).indices.tolist():
        prob = torch.softmax(outputs, -1)[idx].item()
        print('[{idx}] {label:<75} ({p:.2f}%)'.format(idx=idx, label=labels_map[idx], p=prob*100))

In [27]:
# Load Jax weights
npz = np.load(filename)

# Load PyTorch model
model = pytorch_pretrained_vit.ViT(name=name, pretrained=False, load_repr_layer=True)

# Convert weights
new_state_dict = convert(npz, model.state_dict())

# Load into model and test
model.load_state_dict(new_state_dict)

<All keys matched successfully>

In [35]:
check_model(model, name)

[3690] yellow_mountain_saxifrage                                                   (0.07%)
[228] red_fox                                                                     (0.03%)
[7705] amberjack                                                                   (0.02%)


In [30]:
model.pre_logits.weight

Parameter containing:
tensor([[-0.1088,  0.0409,  0.0360,  ..., -0.0716, -0.0018,  0.0037],
        [-0.0945,  0.0493, -0.1079,  ..., -0.0731, -0.0225, -0.1013],
        [ 0.0356, -0.0351,  0.0510,  ...,  0.0867, -0.0274, -0.0638],
        ...,
        [ 0.0978, -0.1415,  0.0287,  ...,  0.0058, -0.0644,  0.0968],
        [ 0.0014,  0.0605, -0.0371,  ..., -0.1093, -0.1687, -0.0141],
        [ 0.0515, -0.1962,  0.0966,  ..., -0.0401, -0.1180, -0.0609]],
       requires_grad=True)

In [31]:
npz['pre_logits/kernel']

array([[-0.10884866, -0.09448911,  0.03560322, ...,  0.09780853,
         0.00137385,  0.05146733],
       [ 0.04090862,  0.04934628, -0.03513117, ..., -0.14147781,
         0.06050842, -0.19619823],
       [ 0.03600169, -0.10787308,  0.05104499, ...,  0.02869395,
        -0.03711622,  0.09662006],
       ...,
       [-0.071619  , -0.07306355,  0.08665656, ...,  0.00580501,
        -0.10929134, -0.04009511],
       [-0.00184033, -0.02247062, -0.02740383, ..., -0.06440727,
        -0.16867375, -0.11800908],
       [ 0.00365812, -0.10132378, -0.06381997, ...,  0.09679291,
        -0.01409991, -0.06087695]], dtype=float32)